In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout
from keras.optimizers import SGD
from keras.utils import np_utils
import scipy.io as sio
import tensorflow as tf
import os

# 读取数据保存到mat
# data = pd.read_csv('./trainnew.csv')
# data = np.array(data)
# label = pd.read_csv('./labelnew.csv')
# label = np.array(label)

# sio.savemat("datanew.mat", {"data": data} )
# sio.savemat("labelnew.mat", {"label": label} )
# print(data.shape)
# print(label.shape)

# 直接从保存的mat中读取数据
train = sio.loadmat("datanew.mat")['data']
label = sio.loadmat("labelnew.mat")['label']
# print(train.shape)
# print(label.shape)

# 标准化训练集
scaler = StandardScaler().fit(train)
scaled_train = scaler.transform(train)

# 交叉验证分类
sss = StratifiedShuffleSplit(test_size=0.2, random_state=0)
for train_index, test_index in sss.split(scaled_train, label):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = scaled_train[train_index], scaled_train[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
# print("X_train.shape:"+str(X_train.shape))
# print("y_train.shape:"+str(y_train.shape))
# print("X_test.shape:"+str(X_test.shape))
# print("y_test.shape:"+str(y_test.shape))

nb_features = 651 # 每一个特征数量 (Ⅱ, Ⅲ, aVF)   
nb_class = 2 # 最后结果

#(20842,1953)

#(20842,651,3)


# 训练集维度变换
X_train_r = np.zeros((len(X_train), nb_features, 3))
X_train_r[:, :, 0] = X_train[:, :nb_features] #0~650
X_train_r[:, :, 1] = X_train[:, nb_features:nb_features*2]# 651~1301
X_train_r[:, :, 2] = X_train[:, nb_features*2:]#1302~end

# 验证集维度变换
X_test_r = np.zeros((len(X_test), nb_features, 3))
X_test_r[:, :, 0] = X_test[:, :nb_features]
X_test_r[:, :, 1] = X_test[:, nb_features:nb_features*2]
X_test_r[:, :, 2] = X_test[:, nb_features*2:]

# Keras model 一维卷积
# 不好的是卷积层和滤波器层数较大
# 不能给很好的正确率
model = Sequential()
model.add(Convolution1D(input_shape=(nb_features, 3), kernel_size=1, filters=256))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(nb_class))
model.add(Activation('softmax'))

y_train = np_utils.to_categorical(y_train, nb_class)
y_test = np_utils.to_categorical(y_test, nb_class)

model.compile(loss='categorical_crossentropy',optimizer = "adam",metrics=['accuracy'])

nb_epoch = 3
model.fit(X_train_r, y_train, epochs=nb_epoch, batch_size=50)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
16673/16673 [==============================] - 33s 2ms/step - loss: 0.0461 - acc: 0.9855
Epoch 2/3
16673/16673 [==============================] - 28s 2ms/step - loss: 0.0047 - acc: 0.9994
Epoch 3/3
16673/16673 [==============================] - 29s 2ms/step - loss: 0.0230 - acc: 0.9952


In [2]:
preds = model.evaluate(x = X_test_r, y = y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

4169/4169 [==============================] - 1s 253us/step
Loss = 0.00863345675159722
Test Accuracy = 0.9992804029743344
